In [1]:
import os

from groq import Groq

In [2]:
groq_key = os.getenv("GROQ_API_KEY")

In [3]:
def load_cvs_from_folder(folder_path):
    cvs = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as file:
                cvs.append(file.read())
    return cvs

folder_path = "cv_cleaned"
cvs = load_cvs_from_folder(folder_path)

FileNotFoundError: [Errno 2] No such file or directory: 'cv_cleaned'

In [4]:
def evaluate_cv(cv_text, job_description):
    prompt = f"""
    Tu es un expert en recrutement et en analyse de CV. 
    Ton rôle est d'évaluer un CV par rapport à une offre d'emploi, puis de générer un CV optimisé.

    Analyse et Extraction des informations du CV :
    - Nom du candidat (si présent)
    - Contact (email, téléphone, LinkedIn s’il est mentionné)
    - Expériences professionnelles (dates, entreprises, responsabilités)
    - Compétences techniques et soft skills
    - Formations et diplômes
    - Projets et réalisations pertinentes

    Évaluation du CV par rapport à l'offre d'emploi :
    - Compare les exigences de l'offre et le contenu du CV.
    - Indique les forces du CV par rapport au poste.
    - Identifie les lacunes : compétences ou expériences manquantes.
    - Suggère des améliorations pour correspondre au mieux à l’offre.

    Génération d’un CV amélioré :
    - Reformule le CV pour mieux correspondre à l’offre.
    - Mets en avant les compétences et expériences clés.
    - Utilise un langage clair et professionnel.
    - Structure le CV dans un format lisible.

    CV du candidat :
    \"\"\"  
    {cv_text}  
    \"\"\"  

    Offre d'emploi : 
    \"\"\"  
    {job_description}  
    \"\"\"  

    Retourne un format structuré avec :
    - [EXTRACTION] Les infos du CV  
    - [ÉVALUATION] Points forts / points faibles  
    - [CV AMÉLIORÉ] Version réécrite du CV  
    """

    client = Groq(api_key=groq_key)
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "Tu es un expert en recrutement."},
            {"role": "user", "content": prompt}
        ],
        model="llama-3.3-70b-versatile",
    )

    return chat_completion.choices[0].message.content



In [9]:
# Dossier des CV
folder_path = "cv_cleaned"
cvs = load_cvs_from_folder(folder_path)
cvs = cvs[:5]

In [10]:
# Description du poste
job_description = "Nous recherchons un développeur Python avec 3 ans d'expérience..."


evaluations = []
for cv in cvs:
    evaluation = evaluate_cv(cv, job_description)
    evaluations.append(evaluation)
    print(f"Evaluation pour le CV :\n{evaluation}\n")

# Sauvegarder les résultats
results = []
for cv, evaluation in zip(cvs, evaluations):
    results.append({
        "cv_text": cv,
        "evaluation": evaluation
    })

with open("results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("Résultats sauvegardés dans results.json")


Evaluation pour le CV :
**[EXTRACTION] Les infos du CV**

* Nom du candidat : Non mentionné
* Contact : Non mentionné
* Expériences professionnelles :
 + Entrepreneur/General Manager (11/2011 - 11/2013)
 + Safety/Training Coordinator (06/1995 - 01/2011)
* Compétences techniques et soft skills :
 + Gestion de l'exploitation aéroportuaire
 + Sécurité et formation
 + Gestion de projet
 + Analyse de données
 + Communication efficace
 + Résolution de problèmes
* Formations et diplômes :
 + Bachelor en administration publique (2007)
 + Associate en administration des affaires (2004)
* Projets et réalisations pertinentes :
 + Mise en place d'une formation en ligne pour réduire les coûts de travail
 + Réduction des incursions sur les pistes d'atterrissage

**[ÉVALUATION] Points forts / points faibles**

* Points forts :
 + Expérience dans la gestion de l'exploitation aéroportuaire et la sécurité
 + Compétences en analyse de données et résolution de problèmes
 + Expérience dans la formation et 